### TensorBoard可以有效的展示Tensorflow在运行过程中的计算图、各种指标随着时间的变化趋势以及训练中使用到的图像等信息。

TensorBoard可以通过Tensorflow在运行过程中输出的日志文件可视化Tensorflow的运行状态。Tensorflow和TensorBoard运行在不同的进程中，TensorBoard会自动读取Tensorflow的日志文件，并呈现当前Tensorflow程序运行的最新状态。

In [1]:
import tensorflow as tf

input1 = tf.constant([1.0,2.0,3.0], name='input1')
input2 = tf.Variable(tf.random_uniform([3]), name='input2')
output = tf.add_n([input1, input2], name='add')

# 生成一个写日志的writer，并将当前的Tensorflow计算图写入日志。Tensorflow提供了多种写日志文件的API。
writer = tf.summary.FileWriter('log/', tf.get_default_graph()) # 旧版本命令为tf.train.SummaryWriter
writer.close()
# 运行完这段代码之后在终端运行 tensorboard --logdir=log/
# 然后在浏览器输入localhost:6006打开

/home/ajialala/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


![未整理时的图](pic/可视化_1.png)

只有add一个节点但是有很多其他像初始化之类的多余的节点，所以看上去比较乱。为了更好的组织可视化效果图中的计算节点，TensorBoard支持通过Tensorflow命名空间来整理可视化效果图上的节点。在Tensorboard的默认视图中，Tensorflow计算图中同一个命名空间下的所有节点会被缩略成一个节点，只有顶层命名空间的节点才会被显示在Tensorboard的可视化界面上。除了之前介绍到tf.variable_scope函数，tf.name_scope函数也提供了命名空间管理功能。以下两个函数的区别：

In [3]:
with tf.variable_scope('foo'):
    a = tf.get_variable('bar', [1])
    print(a.name)
    
with tf.variable_scope('bar'):
    b = tf.get_variable('bar', [1])
    print(b.name)

with tf.name_scope('a'):
    a = tf.Variable([1])
    print(a.name) # tf.Variable生成变量会受tf.name_scope的影响，于是变量名称为a/Variable
    
    a = tf.get_variable('b',[1])
    print(a.name) # tf.get_variable生成变量不会受tf.name_scope的影响，于是变量名称为b，a不出现
    
with tf.name_scope('b'):
    # 因为tf.get_variable生成变量不会受tf.name_scope的影响，这里想获取名称为‘b’的变量，然而这个变量已经被声明了，于是这里会报
    # 重复声明的错误：
    # ValueError: Variable b already exists, disallowed. Did you mean to set reuse=True or reuse=tf.
    # AUTO_REUSE in VarScope? Originally defined at:
    tf.get_variable('b', [1])

ValueError: Variable foo/bar already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-2-ac48d85acbec>", line 2, in <module>
    a = tf.get_variable('bar', [1])
  File "/home/ajialala/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/ajialala/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


用tf.name_scope对第一段代码进行改进

In [1]:
# 将输入定义放入各自的命名空间中
# 此段代码需restart kernel重新运行
import tensorflow as tf
with tf.name_scope('input1'):
    input1 = tf.constant([1.0,2.0,3.0], name='input1')
    
with tf.name_scope('input2'):
    input2 = tf.Variable(tf.random_uniform([3]), name='input2')

output = tf.add_n([input1, input2], name='add')

writer = tf.summary.FileWriter('log/', tf.get_default_graph()) # 旧版本命令为tf.train.SummaryWriter
writer.close()

/home/ajialala/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


![整理后的图](pic/可视化_2.png)

![展开之后](pic/可视化_3.png)